### Incrementales 

### Generar sesion de Spark 

In [1]:
import sys, importlib
sys.path.append(r"C:\Users\DARY\Desktop\Dary\Prueba-BigData\Proyecto-hive\scripts")

import parametrossh
importlib.reload(parametrossh)

from parametrossh import get_params, init_spark, get_sqlserver_connection


In [4]:

# 1. Iniciar sesión de Spark
spark = init_spark(1, "Cargaclientes")
print("✅ SparkSession inicializada")


✅ SparkSession inicializada


In [5]:
# 2. Conexión SQL Server
jdbc_url, properties = get_sqlserver_connection(1)

# 3. Leer tabla Pedidos
df_clientes = spark.read.jdbc(
    url=jdbc_url,
    table="dbo.Clientes",
    properties=properties
)
print("✅ Clientes desde SQL Server")
df_clientes.show(5)

✅ Clientes desde SQL Server


[Stage 0:>                                                          (0 + 1) / 1]

+---------+-----------+--------------------+----------+--------------------+--------------------+
|ClienteID|     Nombre|               Email|  Telefono|          CreateTime|          UpdateTime|
+---------+-----------+--------------------+----------+--------------------+--------------------+
|        1| Juan Pérez|juan.perez@email.com| 089000100|2025-08-24 21:27:...|2025-08-24 21:31:...|
|        2|María López|mar23928.actualic...|0987654321|2025-08-24 21:27:...|2025-09-11 17:22:...|
|        3|Carlos Ruiz|carlos.ruiz@email...|0971122334|2025-08-24 21:27:...|                null|
|        4| Ana Torres|ana.torres@email.com|0965544332|2025-08-24 21:27:...|                null|
|        5|Pedro Gómez|pedro.gomez@email...|0956677889|2025-08-24 21:27:...|                null|
+---------+-----------+--------------------+----------+--------------------+--------------------+
only showing top 5 rows



In [ ]:
#Consulta la ultima fecha de carga de datos
ware_df = spark.read.parquet("hdfs://namenode:8020/bronze/clientes_watermark")
ware_df.show(truncate=False)

[Stage 3:>                                                          (0 + 1) / 1]

+-----------------------+--------+
|last_watermark         |tabla   |
+-----------------------+--------+
|2025-09-11 16:03:17.118|clientes|
+-----------------------+--------+



In [10]:
# Guarda la ultima marca de agua en la ruta HDSF
marca_df= (ware_df
           .filter(ware_df.tabla=="clientes")
           .collect()[0]["last_watermark"])
print("🕒 Última marca de agua usada:", marca_df)

[Stage 6:>                                                          (0 + 1) / 1]

🕒 Última marca de agua usada: 2025-09-11 16:03:17.118000


In [11]:
# 1. Conexión SQL parametrizada
jdbc_url, properties = get_sqlserver_connection(1)

# 2. Construir query incremental
query_clientes_incr = f"""
(
   SELECT *
   FROM dbo.Clientes
   WHERE CreateTime > '{marca_df}'
) AS pedidos_incr
"""

In [12]:
# 3. Leer desde SQL Server
df_incremental = spark.read.jdbc(
    url=jdbc_url,
    table=query_clientes_incr,
    properties=properties
)

# 4. Validar
if df_incremental.rdd.isEmpty():
    print("⚠️ No hay registros nuevos después de", df1)
else:
    print("✅ Nuevos registros encontrados:", df_incremental.count())
    df_incremental.show(5)

✅ Nuevos registros encontrados: 1
+---------+----------+--------------------+----------+--------------------+----------+
|ClienteID|    Nombre|               Email|  Telefono|          CreateTime|UpdateTime|
+---------+----------+--------------------+----------+--------------------+----------+
|       16|Guale Dary|Guale.Dary@email.com|0991227667|2025-09-30 15:13:...|      null|
+---------+----------+--------------------+----------+--------------------+----------+



In [13]:


# 4. Guardar los nuevos registros en Bronze
df_incremental.write.mode("append").parquet("hdfs://namenode:8020/bronze/clientes")
print("✅ Nuevos registros cargados en Bronze.")

[Stage 11:>                                                         (0 + 1) / 1]

✅ Nuevos registros cargados en Bronze.


In [14]:
df_clie = spark.read.parquet("hdfs://namenode:8020/bronze/clientes")
df_clie.show(30)

+---------+--------------+--------------------+----------+--------------------+--------------------+
|ClienteID|        Nombre|               Email|  Telefono|          CreateTime|          UpdateTime|
+---------+--------------+--------------------+----------+--------------------+--------------------+
|        1|    Juan Pérez|juan.perez@email.com| 089000100|2025-08-24 21:27:...|2025-08-24 21:31:...|
|        2|   María López|maria22.nueva@ema...|0987654321|2025-08-24 21:27:...|2025-08-24 21:31:...|
|        3|   Carlos Ruiz|carlos.ruiz@email...|0971122334|2025-08-24 21:27:...|                null|
|        4|    Ana Torres|ana.torres@email.com|0965544332|2025-08-24 21:27:...|                null|
|        5|   Pedro Gómez|pedro.gomez@email...|0956677889|2025-08-24 21:27:...|                null|
|        7|     leo Pérez| leo.perez@email.com|0991234522|2025-08-29 19:50:...|                null|
|       16|    Guale Dary|Guale.Dary@email.com|0991227667|2025-09-30 15:13:...|            

In [15]:
from pyspark.sql.functions import max as spark_max, lit

marca_df = df_clie.agg(spark_max("CreateTime").alias("last_watermark")) \
                  .withColumn("tabla", lit("clientes"))


In [16]:
marca_df.show(truncate=False)

+-----------------------+--------+
|last_watermark         |tabla   |
+-----------------------+--------+
|2025-09-30 15:13:53.417|clientes|
+-----------------------+--------+



In [17]:
marca_df.write.mode("overwrite").parquet("hdfs://namenode:8020/bronze/clientes_watermark")

In [18]:
ware_df = spark.read.parquet("hdfs://namenode:8020/bronze/clientes_watermark")
ware_df.show(truncate=False)

+-----------------------+--------+
|last_watermark         |tabla   |
+-----------------------+--------+
|2025-09-30 15:13:53.417|clientes|
+-----------------------+--------+



In [19]:
spark.stop()